---
title: "Midterm Project"
author: "Alyssa Horn"
toc: true
number-sections: true
self-contained: true
highlight-style: pygments
format: 
  html: 
    code-fold: true
    html-math-method: katex
  pdf: 
    geometry: 
      - top=30mm
      - left=20mm
---


## Notes
* Cleaning
    * missing zipcode not as big an issue for this dataset
* Exploration
    * Use leaflet like the homework but using different symbols
    * response time - someone needs to look at it and close it - from created date to closed date
    * 2 descriptors: SF and CB. Might want to do the comparison by borough. These are right skewed data. Might need to use a different scale?
    * hypothesis tests
      * Difference in borough: 2-way ANOVA, justify whatever test used
    * Binary variable: should be similar to what we have in iii
* Modeling the count
    * i. should get 174
    * Use zipcode as key to merge
    * lag: maybe if there is more rain - then there are more complaints the next day?
    * Neg binomial is def better because mean and variance do not need to be the same

# Data Cleaning

## Import the data, rename the columns with our preferred styles.

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

# Load data
file_path = "nycflood2024.csv" 
df = pd.read_csv(file_path)

# Rename columns
df.columns = df.columns.str.lower().str.replace(" ", "_")

df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,63574884,12/31/2024 11:05:00 PM,01/01/2025 05:55:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11434.0,177-37 135 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.675681,-73.762969,"(40.67568056757403, -73.76296916912234)"
1,63580185,12/31/2024 11:02:00 PM,01/01/2025 10:30:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11219.0,4102 14 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.638098,-73.985346,"(40.63809797146557, -73.98534616586583)"
2,63573084,12/31/2024 11:02:00 PM,01/01/2025 06:20:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11361.0,35-34 CORPORAL KENNEDY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.767455,-73.778079,"(40.767454543462755, -73.77807940403599)"
3,63581809,12/31/2024 10:35:00 PM,01/01/2025 11:20:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11219.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636495,-73.987000,"(40.636495214873584, -73.98700030399515)"
4,63578460,12/31/2024 10:11:00 PM,01/02/2025 12:45:00 PM,DEP,Department of Environmental Protection,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),NaN,10305.0,225 HYLAN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.612337,-74.069589,"(40.612337196854604, -74.06958876337421)"
